In [ ]:
# # Colab에 Mecab 설치
# !git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
# %cd Mecab-ko-for-Google-Colab
# !bash install_mecab-ko_on_colab190912.sh

In [ ]:
# # 위의 코드가 작동하지 않을 때 대안 코드 1 실행
# !pip install konlpy
# !pip install mecab-python
# !bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

#NameError: name 'Tagger' is not defined 오류 발생 시 런타임을 재실행 해주세요.

In [ ]:
# 한글 자모 단위 처리 패키지 설치
!pip install hgtk

In [ ]:
# fasttext 설치
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!make
!pip install .

fatal: destination path 'fastText' already exists and is not an empty directory.
/content/fastText
make: Nothing to be done for 'opt'.
Processing /content/fastText
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fasttext: filename=fasttext-0.9.2-cp311-cp311-linux_x86_64.whl size=4313389 sha256=50bd651dc3b746181e069237c89e8783e6c2e5491592dd8d9f0f4638757394b1
  Stored in directory: /tmp/pip-ephem-wheel-cache-2pc4n5cv/wheels/1e/bc/93/e4d717605b2816cf84bf6796e766c05fcc2fd4feb3f170fdf3
Successfully built fasttext
  Attempting uninstall: fasttext
    Found existing installation: fasttext 0.9.2
    Uninstalling fasttext-0.9.2:
      Successfully uninstalled fasttext-0.9.2


In [ ]:
#hgtk 설치
!pip install hgtk

## 1. 데이터 로드

In [ ]:
import re
import pandas as pd
import urllib.request
from tqdm import tqdm
import hgtk

In [ ]:
# 네이버 쇼핑 리뷰
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/naver_shopping.txt", filename="ratings_total.txt")

('ratings_total.txt', <http.client.HTTPMessage at 0x7fb802bfc910>)

In [ ]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력

전체 리뷰 개수 : 200000


In [ ]:
total_data.head()

,ratings,reviews
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


## 2. hgtk 튜토리얼



word embedding이 단어 단위의 임베딩이었다면, character embedding은 문자 단위의 임베딩입니다. 한국어를 character embedding할 수 있는 것이 바로 자음 모음 분리기 hgtk입니다.

 영어는 하나의 알파벳(52자)를 기준으로 character embedding을 하지만, 한국어에서 하나의 음절별로 character embedding을 하면 11172개의 음절이 있기 때문에 계산량이 너무 많습니다. 따라서 그보다 작은 단위인 자음,모음으로 분리하는 것입니다.

 >참고 repo: https://github.com/bluedisk/hangul-toolkit

In [ ]:
# 한글인지 체크
print(hgtk.checker.is_hangul('ㄱ'))
print(hgtk.checker.is_hangul('12'))
print(hgtk.checker.is_hangul('a'))

True
False
False


In [ ]:
# 음절을 초성, 중성, 종성으로 분해
print(hgtk.letter.decompose('남'))
# 초성, 중성, 종성을 하나의 음절로 결합
print(hgtk.letter.compose('ㄴ', 'ㅏ', 'ㅁ'))

('ㄴ', 'ㅏ', 'ㅁ')
남


In [ ]:
# 결합할 수 없는 상황에서는 에러 발생
try:
  hgtk.letter.compose('ㄴ', 'ㅁ', 'ㅁ') # 중성이 없는 경우
except:
  print('에러 발생')

에러 발생


## 3. 데이터 전처리

![image.png](attachment:image.png)

In [ ]:
def word_to_jamo(token):
  def to_special_token(jamo): # 경우에 따라 초, 중, 종성이 다 있는 게 아닌 경우도 있다. 이 경우 -를 반환해주는 함수
    if not jamo:
      return '-'
    else:
      return jamo

  decomposed_token = ''
  for char in token:
    try:
      # char(음절)을 초성, 중성, 종성으로 분리
      cho, jung, jong = hgtk.letter.decompose(char)

      # 자모가 빈 문자일 경우 특수문자 -로 대체
      cho = to_special_token(cho)
      jung = to_special_token(jung)
      jong = to_special_token(jong)
      decomposed_token = decomposed_token + cho + jung + jong

    # 만약 char(음절)이 한글이 아닐 경우 자모를 나누지 않고 추가
    except Exception as exception:
      if type(exception).__name__ == 'NotHangulException':
        decomposed_token += char

  # 단어 토큰의 자모 단위 분리 결과를 추가
  return decomposed_token

In [ ]:
print(word_to_jamo('남동생'))
print(word_to_jamo('야구')) # 야구의 경우 종성이 없으므로 종성 부분을 -로 반환

ㄴㅏㅁㄷㅗㅇㅅㅐㅇ
ㅇㅑ-ㄱㅜ-


In [ ]:
print(mecab.morphs('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['선물', '용', '으로', '빨리', '받', '아서', '전달', '했어야', '하', '는', '상품', '이', '었', '는데', '머그', '컵', '만', '와서', '당황', '했', '습니다', '.']


In [ ]:
# mecab으로 형태소를 분리해주고 그 형태소마다 각각 자음모음을 분리해주는 함수
def tokenize_by_jamo(s):
    return [word_to_jamo(token) for token in mecab.morphs(s)]

In [ ]:
print(tokenize_by_jamo('선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다.'))

['ㅅㅓㄴㅁㅜㄹ', 'ㅇㅛㅇ', 'ㅇㅡ-ㄹㅗ-', 'ㅃㅏㄹㄹㅣ-', 'ㅂㅏㄷ', 'ㅇㅏ-ㅅㅓ-', 'ㅈㅓㄴㄷㅏㄹ', 'ㅎㅐㅆㅇㅓ-ㅇㅑ-', 'ㅎㅏ-', 'ㄴㅡㄴ', 'ㅅㅏㅇㅍㅜㅁ', 'ㅇㅣ-', 'ㅇㅓㅆ', 'ㄴㅡㄴㄷㅔ-', 'ㅁㅓ-ㄱㅡ-', 'ㅋㅓㅂ', 'ㅁㅏㄴ', 'ㅇㅘ-ㅅㅓ-', 'ㄷㅏㅇㅎㅘㅇ', 'ㅎㅐㅆ', 'ㅅㅡㅂㄴㅣ-ㄷㅏ-', '.']


In [ ]:
# 리뷰 데이터의 reviews 컬럼만을 가져와서 자모 분리
tokenized_data = []

for sample in tqdm(total_data['reviews'].to_numpy()):
    tokenzied_sample = tokenize_by_jamo(sample) # 자소 단위 토큰화
    tokenized_data.append(tokenzied_sample)

100%|██████████| 200000/200000 [01:02<00:00, 3216.06it/s]


In [ ]:
print(len(tokenized_data))
print("전처리 전:", total_data['reviews'][1])
print("전처리 후:", tokenized_data[1])

200000
전처리 전: 택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
전처리 후: ['ㅌㅐㄱㅂㅐ-', 'ㄱㅏ-', 'ㅇㅓㅇㅁㅏㅇ', 'ㅇㅣ-', 'ㄴㅔ-', 'ㅇㅛㅇ', 'ㅈㅓ-ㅎㅢ-', 'ㅈㅣㅂ', 'ㅁㅣㅌ', 'ㅇㅔ-', 'ㅊㅡㅇ', 'ㅇㅔ-', 'ㅁㅏㄹ', 'ㄷㅗ-', 'ㅇㅓㅄㅇㅣ-', 'ㄴㅘ-ㄷㅜ-', 'ㄱㅗ-', 'ㄱㅏ-', 'ㄱㅗ-']


단어를 자모 분리한 것을 역으로 하여 자모 상태를 단어로 다시 결합시키는 함수도 정의해봅시다. 이는 단어의 코사인 유사도를 평가할 때 자모 분리가 된 상태가 아니라 단어 상태로 편리하게 보기 위함입니다.

In [ ]:
def jamo_to_word(jamo_sequence):
  tokenized_jamo = []
  index = 0

  # 1. 초기 입력
  # jamo_sequence = 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

  while index < len(jamo_sequence):
    # 문자가 한글(정상적인 자모)이 아닐 경우
    if not hgtk.checker.is_hangul(jamo_sequence[index]):
      tokenized_jamo.append(jamo_sequence[index])
      index = index + 1

    # 문자가 정상적인 자모라면 초성, 중성, 종성을 하나의 토큰으로 간주.
    else:
      tokenized_jamo.append(jamo_sequence[index:index + 3])
      index = index + 3

  # 2. 자모 단위 토큰화 완료
  # tokenized_jamo : ['ㄴㅏㅁ', 'ㄷㅗㅇ', 'ㅅㅐㅇ']

  word = ''
  try:
    for jamo in tokenized_jamo:

      # 초성, 중성, 종성의 묶음으로 추정되는 경우
      if len(jamo) == 3:
        if jamo[2] == "-":
          # 종성이 존재하지 않는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1])
        else:
          # 종성이 존재하는 경우
          word = word + hgtk.letter.compose(jamo[0], jamo[1], jamo[2])
      # 한글이 아닌 경우
      else:
        word = word + jamo

  # 복원 중(hgtk.letter.compose) 에러 발생 시 초기 입력 리턴.
  # 복원이 불가능한 경우 예시) 'ㄴ!ㅁㄷㅗㅇㅅㅐㅇ'
  except Exception as exception:
    if type(exception).__name__ == 'NotHangulException':
      return jamo_sequence

  # 3. 단어로 복원 완료
  # word : '남동생'

  return word

## 4. FastText

In [ ]:
import fasttext

fasttext를 실행하기에 앞서 훈련 대상인 단어들을 txt 파일로 준비해둬야 합니다. 따라서 `tokenized_data.txt`라는 파일을 쓰기 모드(w)로 생성해주고 앞서 전처리한 `tokenized_data`를 입력해줍니다.

In [ ]:
with open('tokenized_data.txt', 'w') as out:
  for line in tqdm(tokenized_data, unit=' line'):
    out.write(' '.join(line) + '\n')

100%|██████████| 200000/200000 [00:00<00:00, 354154.28 line/s]


아래처럼 `train_unsupeviesd` 함수는 훈련을 시켜주는 함수입니다. 인자로 훈련할 단어가 담긴 txt 파일을 지정하고 model을 `cbow`나 `skipgram` 중에 하나를 고르면 됩니다.

In [ ]:
model = fasttext.train_unsupervised('tokenized_data.txt', model='cbow')

In [ ]:
model.save_model("fasttext.bin")

In [ ]:
model = fasttext.load_model("fasttext.bin")

In [ ]:
model[word_to_jamo('남동생')] # 'ㄴㅏㅁㄷㅗㅇㅅㅐㅇ'

array([-0.1783078 ,  0.790016  ,  0.77795655,  0.36818233,  0.46228698,
       -0.3720079 ,  0.27041084, -0.3125775 ,  0.1700772 , -0.12752165,
        0.2005062 ,  0.70349866, -0.32784566,  0.40647647,  0.8274091 ,
       -0.2697191 ,  0.3788647 ,  0.13849564,  0.11105179, -0.17119847,
        0.01293096, -0.7637373 , -0.05854314, -0.48128685, -0.47419748,
       -0.49593917,  0.37822896,  0.4758871 ,  0.73222214,  0.14633812,
       -0.04399045, -1.2377392 ,  0.3140675 ,  0.7687821 ,  0.6565598 ,
       -0.33240208,  0.48727417, -0.5810977 ,  0.11862421,  0.9968714 ,
        0.7153069 ,  0.57084846, -0.43699387,  1.2362758 ,  0.06930456,
       -0.21719797,  0.8579173 ,  0.06309998,  0.04849518, -0.12291124,
       -0.00943801, -0.01136472,  0.755355  , -0.01285179,  0.14335221,
       -0.62276447,  0.10659599,  0.2786796 , -0.09709349,  0.34252763,
       -0.68762666,  0.17060655,  0.5063953 ,  0.1175071 , -0.01823661,
       -0.63972175, -0.45963728,  0.01354522, -0.14149809, -1.03

`get_nearest_neighbors` 함수를 사용하여 '남동생'이라는 단어와 가장 유사도가 높은 단어들(자모 분리된 상태)을 k개만큼 출력해줍니다.

In [ ]:
model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)

[(0.8852669596672058, 'ㄷㅗㅇㅅㅐㅇ'),
 (0.8423430323600769, 'ㄴㅏㅁㅊㅣㄴ'),
 (0.7675397396087646, 'ㅅㅐㅇㅇㅣㄹ'),
 (0.7460188269615173, 'ㅊㅣㄴㄱㅜ-'),
 (0.743413507938385, 'ㄴㅏㅁㅍㅕㄴ'),
 (0.7353487014770508, 'ㅈㅗ-ㅋㅏ-'),
 (0.7348003387451172, 'ㄴㅏㅁㅇㅏ-'),
 (0.7188143730163574, 'ㄴㅏㄴㅅㅐㅇ'),
 (0.7105963826179504, 'ㅈㅜㅇㅎㅏㄱㅅㅐㅇ'),
 (0.6962770819664001, 'ㅎㅏㄱㅅㅐㅇ')]

앞서 만든 `jamo_to_word`로 가독성이 좋게 출력해봅시다.

In [ ]:
def transform(word_sequence):
  return [(jamo_to_word(word), similarity) for (similarity, word) in word_sequence]

In [ ]:
print(transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))
print(transform(model.get_nearest_neighbors(word_to_jamo('구매'), k=10)))
print(transform(model.get_nearest_neighbors(word_to_jamo('배달'), k=10)))

[('동생', 0.8852669596672058), ('남친', 0.8423430323600769), ('생일', 0.7675397396087646), ('친구', 0.7460188269615173), ('남편', 0.743413507938385), ('조카', 0.7353487014770508), ('남아', 0.7348003387451172), ('난생', 0.7188143730163574), ('중학생', 0.7105963826179504), ('학생', 0.6962770819664001)]
[('구매처', 0.8410975337028503), ('구입', 0.7884979248046875), ('주문', 0.7431554794311523), ('주문건', 0.672484815120697), ('구명조끼', 0.6199132800102234), ('헤매', 0.616483211517334), ('주문서', 0.6153122782707214), ('구매자', 0.5983448028564453), ('구메', 0.5914770364761353), ('재', 0.5779432654380798)]
[('배송지', 0.8157444596290588), ('깨달', 0.7954130172729492), ('매달', 0.7811452746391296), ('메달', 0.7469878196716309), ('택배', 0.7413893342018127), ('백배', 0.7330837845802307), ('운송장', 0.7324768900871277), ('배소', 0.7289184331893921), ('송장', 0.7288572788238525), ('배공', 0.7204813361167908)]


## 5. Word2Vec

![image.png](attachment:image.png)
이제 word2vec를 사용하여 자모 단위로 분리하는 것이 아닌 단어 단위로 분리하여 임베딩 벡터를 생성해볼 것입니다.

In [ ]:
# 간단하게 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

tokenized_data2 = []
for sentence in tqdm(total_data['reviews'].to_list()):
    tokenized_sentence = mecab.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data2.append(stopwords_removed_sentence)

100%|██████████| 200000/200000 [00:22<00:00, 9081.70it/s] 


In [ ]:
print("word2vec용 데이터:", tokenized_data2[0])
print("fasttext용 데이터:", tokenized_data[0])

word2vec용 데이터: ['배공', '빠르', '고', '굿']
fasttext용 데이터: ['ㅂㅐ-ㄱㅗㅇ', 'ㅃㅏ-ㄹㅡ-', 'ㄱㅗ-', 'ㄱㅜㅅ']


In [ ]:
from gensim.models import Word2Vec

model2 = Word2Vec(sentences = tokenized_data2, vector_size = 1000, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
# 완성된 임베딩 매트릭스의 크기 확인
# 단어의 총 개수는 15005개이고 벡터 차원은 1000으로 축소되었다.
model2.wv.vectors.shape

(15004, 1000)

## 6. FastText와 Word2Vec 결과 비교

### **Q1) 남동생과 주문 두 단어를 input으로 넣고 결과를 비교한 뒤 해석해보세요**

*(Hint: 유사도, 단어의 의미, 단어의 생김새 등을 고려해 볼 수 있습니다)*

In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('남동생'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("남동생"))

FastText 유사도: [('동생', 0.8852669596672058), ('남친', 0.8423430323600769), ('생일', 0.7675397396087646), ('친구', 0.7460188269615173), ('남편', 0.743413507938385), ('조카', 0.7353487014770508), ('남아', 0.7348003387451172), ('난생', 0.7188143730163574), ('중학생', 0.7105963826179504), ('학생', 0.6962770819664001)]
Word2Vec 유사도: [('후배', 0.6980202198028564), ('언니', 0.6790263652801514), ('결혼', 0.673446774482727), ('조카', 0.667224109172821), ('오빠', 0.6605604887008667), ('동생', 0.6599416732788086), ('남친', 0.6537732481956482), ('외할머니', 0.6458012461662292), ('친구', 0.6401689052581787), ('개업', 0.6359803080558777)]


In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('주문'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("주문"))

FastText 유사도: [('주문건', 0.9121477603912354), ('주문서', 0.8553550839424133), ('주문자', 0.7449468374252319), ('구매', 0.7431555390357971), ('주무', 0.7370957136154175), ('주무시', 0.7262292504310608), ('구매처', 0.7238249182701111), ('구입', 0.7236705422401428), ('시킨', 0.6983362436294556), ('시켰었', 0.6900933384895325)]
Word2Vec 유사도: [('구입', 0.8219693899154663), ('구매', 0.8217630386352539), ('선택', 0.6287480592727661), ('시켰', 0.5652076601982117), ('장만', 0.56418776512146), ('결제', 0.5533924698829651), ('준비', 0.5414609313011169), ('시킨', 0.5280566811561584), ('신청', 0.5244446396827698), ('도착', 0.5234110355377197)]


**Your Inference: FastText가 아무래도 단어를 자모단위로 나눠서 학습하다보니, 해당 단어를 어근으로 갖는 관련있는 단어 혹은 단어를 분리해서 관련있는 것들을 위주로 출력해서 더 좋은 결과가 나온 것 같다. 그에 반해 word2vec은 말그대로 어원에 뜻이나 이런 것들을 고려하기보단 동의어나 반의어 이런식의 관계를 가지고 유사도를 측정해 결과를 도출한 것 같다.

### **Q2) Fasttext가 Word2Vec보다 항상 성능이 나은지 다양한 input을 넣어서 시도해보세요**

In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('기쁨'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("기쁨"))

FastText 유사도: [('기쁘', 0.7970003485679626), ('기엽', 0.7797866463661194), ('기쁜', 0.7435135841369629), ('기뻐요', 0.6915820240974426), ('기쁩니다', 0.673945426940918), ('기특', 0.6731407642364502), ('기격', 0.668286144733429), ('기겁', 0.6577785015106201), ('기편', 0.6396363377571106), ('기술', 0.623359739780426)]
Word2Vec 유사도: [('훌', 0.6915708184242249), ('도서관', 0.6895105242729187), ('편안히', 0.6838011741638184), ('통발', 0.6829121708869934), ('젛아여', 0.6794799566268921), ('~^^*', 0.6785073280334473), ('정석', 0.6780622601509094), ('ㅡㄷ', 0.6758782863616943), ('햅', 0.6703732013702393), ('ㅇㅊ', 0.6687451004981995)]


In [ ]:
print("FastText 유사도:", transform(model.get_nearest_neighbors(word_to_jamo('축구'), k=10)))
print("Word2Vec 유사도:", model2.wv.most_similar("축구"))

FastText 유사도: [('탁구', 0.7755910754203796), ('배수구', 0.741094172000885), ('방구', 0.7394518852233887), ('욕구', 0.7347831726074219), ('행구', 0.7081165313720703), ('동구', 0.7047176957130432), ('복구', 0.6977302432060242), ('통풍구', 0.6869428753852844), ('침구', 0.6852038502693176), ('체구', 0.6841150522232056)]
Word2Vec 유사도: [('유치원생', 0.7541922926902771), ('신음', 0.7505425214767456), ('라이더', 0.7497990131378174), ('ㅍㄱ', 0.749634325504303), ('앞둔', 0.7482443451881409), ('도면', 0.7462185025215149), ('트레이닝', 0.7455251216888428), ('봄여름', 0.7446984052658081), ('한파', 0.7446256875991821), ('밥솥', 0.744231104850769)]


**Your Inference:정확도에 있어서 FastText가 훨씬 높은 건 맞는 것 같다. 하지만, 위에 예시로 봤듯이 기쁘 이런식에 단어가 아닌 것들도 벡터값 안에 들어가있는 것들 보면 전처리 단계에서 개선을 하는 방식이 추가적으로 필요해보인다.